In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

import matplotlib.pyplot as plt
import time

import data_splitter as ds
import data_loader as dl
import feature_extraction as fex
import run_model as rm
import eval_model as ev

2023-04-15 11:40:11.899858: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 11:40:19.900049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import subprocess

try:
    subprocess.check_output('nvidia-smi')
    print('Nvidia GPU detected!')
except Exception: # this command not being found can raise quite a few different errors depending on the configuration
    print('No Nvidia GPU in system!')

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

In [3]:
KERASTUNER_TUNER_ID="chief"
KERASTUNER_ORACLE_IP="127.0.0.1"
KERASTUNER_ORACLE_PORT="8000"

In [4]:
splitter = ds.DataSplitter(verbose=True)

Total patient_id in training set:  9530
Total patient_id in test set:  2383
Total image_id in training set:  43767
Total image_id in test set:  10939
Total patient_id in training set:  7624
Total patient_id in calibration set:  1906
Total image_id in training set:  35003
Total image_id in calibration set:  8764


In [5]:
# splitter.trainset is the training set in format of {patient_id: [image_id, ...]}
train_ids = splitter.trainset.keys()
label_by_img = splitter.labels

# getting cross-validation folds from training set
RANDOM_STATE = 42
cv_patient, cv_img = splitter.get_cv(n_splits=5, random_state=RANDOM_STATE)

Splitting training set into 5 stratified k-folds...
--Fold:  0
----Total patient_id in training set:  6099
----Total image_id in training set:  28031
----Total patient_id in validation set:  1525
----Total image_id in validation set:  6972
--Fold:  1
----Total patient_id in training set:  6099
----Total image_id in training set:  28016
----Total patient_id in validation set:  1525
----Total image_id in validation set:  6987
--Fold:  2
----Total patient_id in training set:  6099
----Total image_id in training set:  27937
----Total patient_id in validation set:  1525
----Total image_id in validation set:  7066
--Fold:  3
----Total patient_id in training set:  6099
----Total image_id in training set:  28025
----Total patient_id in validation set:  1525
----Total image_id in validation set:  6978
--Fold:  4
----Total patient_id in training set:  6100
----Total image_id in training set:  28003
----Total patient_id in validation set:  1524
----Total image_id in validation set:  7000


In [6]:
metadata = pd.read_csv('train.csv')

# Train from scratch

### Image Resolution 1024 x 1024

In [7]:
# inception from scratch
IMG_SIZE = (1024, 1024, 1)

inception = keras.applications.InceptionV3(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),  # should have exactly 3 inputs channels because it is pre-trained on RBG images
    include_top=False
    )

inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024, 1024,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 511, 511, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 511, 511, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

                                                                                                  
 average_pooling2d (AveragePool  (None, 125, 125, 19  0          ['max_pooling2d_1[0][0]']        
 ing2D)                         2)                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 125, 125, 64  12288       ['max_pooling2d_1[0][0]']        
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 125, 125, 64  76800       ['activation_6[0][0]']           
                                )                                                                 
                                                                                                  
 conv2d_10

                                                                                                  
 conv2d_18 (Conv2D)             (None, 125, 125, 64  16384       ['average_pooling2d_1[0][0]']    
                                )                                                                 
                                                                                                  
 batch_normalization_12 (BatchN  (None, 125, 125, 64  192        ['conv2d_12[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_normalization_14 (BatchN  (None, 125, 125, 64  192        ['conv2d_14[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_nor

                                                                                                  
 batch_normalization_25 (BatchN  (None, 125, 125, 64  192        ['conv2d_25[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_19 (Activation)     (None, 125, 125, 64  0           ['batch_normalization_19[0][0]'] 
                                )                                                                 
                                                                                                  
 activation_21 (Activation)     (None, 125, 125, 64  0           ['batch_normalization_21[0][0]'] 
                                )                                                                 
                                                                                                  
 activatio

 activation_31 (Activation)     (None, 62, 62, 128)  0           ['batch_normalization_31[0][0]'] 
                                                                                                  
 activation_36 (Activation)     (None, 62, 62, 128)  0           ['batch_normalization_36[0][0]'] 
                                                                                                  
 conv2d_32 (Conv2D)             (None, 62, 62, 128)  114688      ['activation_31[0][0]']          
                                                                                                  
 conv2d_37 (Conv2D)             (None, 62, 62, 128)  114688      ['activation_36[0][0]']          
                                                                                                  
 batch_normalization_32 (BatchN  (None, 62, 62, 128)  384        ['conv2d_32[0][0]']              
 ormalization)                                                                                    
          

                                                                                                  
 conv2d_47 (Conv2D)             (None, 62, 62, 160)  179200      ['activation_46[0][0]']          
                                                                                                  
 batch_normalization_42 (BatchN  (None, 62, 62, 160)  480        ['conv2d_42[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_47 (BatchN  (None, 62, 62, 160)  480        ['conv2d_47[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_42 (Activation)     (None, 62, 62, 160)  0           ['batch_normalization_42[0][0]'] 
          

                                                                                                  
 batch_normalization_57 (BatchN  (None, 62, 62, 160)  480        ['conv2d_57[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_52 (Activation)     (None, 62, 62, 160)  0           ['batch_normalization_52[0][0]'] 
                                                                                                  
 activation_57 (Activation)     (None, 62, 62, 160)  0           ['batch_normalization_57[0][0]'] 
                                                                                                  
 average_pooling2d_5 (AveragePo  (None, 62, 62, 768)  0          ['mixed5[0][0]']                 
 oling2D)                                                                                         
          

                                                                                                  
 activation_67 (Activation)     (None, 62, 62, 192)  0           ['batch_normalization_67[0][0]'] 
                                                                                                  
 average_pooling2d_6 (AveragePo  (None, 62, 62, 768)  0          ['mixed6[0][0]']                 
 oling2D)                                                                                         
                                                                                                  
 conv2d_60 (Conv2D)             (None, 62, 62, 192)  147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_63 (Conv2D)             (None, 62, 62, 192)  258048      ['activation_62[0][0]']          
                                                                                                  
 conv2d_68

 mixed8 (Concatenate)           (None, 30, 30, 1280  0           ['activation_71[0][0]',          
                                )                                 'activation_75[0][0]',          
                                                                  'max_pooling2d_3[0][0]']        
                                                                                                  
 conv2d_80 (Conv2D)             (None, 30, 30, 448)  573440      ['mixed8[0][0]']                 
                                                                                                  
 batch_normalization_80 (BatchN  (None, 30, 30, 448)  1344       ['conv2d_80[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_80 (Activation)     (None, 30, 30, 448)  0           ['batch_normalization_80[0][0]'] 
          

 batch_normalization_89 (BatchN  (None, 30, 30, 448)  1344       ['conv2d_89[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_89 (Activation)     (None, 30, 30, 448)  0           ['batch_normalization_89[0][0]'] 
                                                                                                  
 conv2d_86 (Conv2D)             (None, 30, 30, 384)  786432      ['mixed9[0][0]']                 
                                                                                                  
 conv2d_90 (Conv2D)             (None, 30, 30, 384)  1548288     ['activation_89[0][0]']          
                                                                                                  
 batch_normalization_86 (BatchN  (None, 30, 30, 384)  1152       ['conv2d_86[0][0]']              
 ormalizat

In [8]:
# base_model.summary()
keras.utils.plot_model(inception, show_shapes=True)

In [8]:
train_img_ids = cv_img[0]['train']
val_img_ids = cv_img[0]['validate']

In [10]:
BATCH_SIZE = 1*32
IMG_SIZE = (1024, 1024, 1)
RANDOM_STATE = 42 # the random state that used to create cv splits

BASEPATH = '/scratch/siads699w23_class_root/siads699w23_class/irinb/train'

train_gen, val_gen = rm.get_train_val_generators(
        train_img_ids,
        val_img_ids,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
    
        from_numpy=True,
        basepath=BASEPATH,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
    
        shuffle=True,
        normalize=(0, 1),
        feature_extractor='CNN',
        CNN_preprocess = keras.applications.inception_v3.preprocess_input
        )

In [10]:
print(len(train_gen), len(val_gen))

875 217


In [11]:
X, y = train_gen[0]
print(X.shape, y.shape)

(32, 1024, 1024, 3) (32,)


In [12]:
inception.output.shape[-3:]

TensorShape([30, 30, 2048])

## Use initial bias and class weight

In [9]:
pos = metadata[metadata.cancer==1].image_id.nunique()
neg = metadata[metadata.cancer==0].image_id.nunique()
initial_bias = np.log([pos/neg])
print(initial_bias)

[-3.83388407]


In [10]:
initial_bias = tf.keras.initializers.Constant()

model_layers = keras.Sequential([
    keras.layers.Input(inception.output.shape[-3:]),
    keras.layers.GlobalAvgPool2D(),
    keras.layers.Dense(1, activation='sigmoid', bias_initializer=initial_bias)
])

model_layers.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 2,049
Trainable params: 2,049
Non-trainable params: 0
_________________________________________________________________


In [11]:
pos = metadata[metadata.cancer==1].image_id.nunique()
neg = metadata[metadata.cancer==0].image_id.nunique()
total = metadata[metadata.patient_id.isin(splitter.trainset.keys())].image_id.nunique()

# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.33
Weight for class 1: 15.11


In [ ]:
%load_ext tensorboard

In [16]:
from keras.callbacks import TensorBoard
import os

log_basepath = '/scratch/siads699w23_class_root/siads699w23_class/irinb/logs_'
model_name = 'rsna_1024_bias_weight_adam-lr-1e-default'

model_log_path = log_basepath + model_name
os.makedirs(model_log_path, exist_ok=True)

tensorboard = TensorBoard(
    log_dir=model_log_path,
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq='epoch'
)

In [12]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [13]:
model = rm.make_or_restore_model(
    model_layers,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1]),
    augment_layers=True,
    base_model=inception,
    trainable=True,
    loss_func = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics=METRICS,
    checkpoint_path='models/inception_rsna_checkpoint/adam'
)

model.summary()

Checkpoint path:  models/inception_rsna_checkpoint/adam
---Checkpoint folder already exists
Creating a new model...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 1024, 1024, 3)     0         
                                                                 
 inception_v3 (Functional)   (None, 30, 30, 2048)      21802784  
                                                                 
 sequential (Sequential)     (None, 1)                 2049      
                                                                 
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
start = time.time()

EPOCH = 100

early_stopping = tf.keras.callbacks.EarlyStopping('val_loss', patience=5) # very slow training, never finishes
CHECKPOINT_PATH = 'models/inception_rsna_checkpoint/rsna_1024_bias_weights_adam-lr-default-{epoch:02d}-prc{val_prc:.2f}-tp{val_tp}-loss{val_loss:.2f}.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = CHECKPOINT_PATH,
    save_weights_only = True,
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    save_freq = 'epoch',
    verbose = 1
)

history = model.fit(
    train_gen,
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint, tensorboard],
    use_multiprocessing = True,
    workers = 32,
    validation_data=val_gen,
    class_weight=class_weight,
    verbose = 1
    )


stop = time.time()
print('>>> Training time: ', (stop - start))  

2023-04-09 23:00:31.011895: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [2]
	 [[{{node Placeholder/_3}}]]


Epoch 1/100
875/875 [==============================] - ETA: 0s - loss: 0.4467 - tp: 221.0000 - fp: 10115.0000 - tn: 17306.0000 - fn: 358.0000 - accuracy: 0.6260 - precision: 0.0214 - recall: 0.3817 - auc: 0.4996 - prc: 0.0209 

2023-04-10 07:25:01.577859: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 171825.51562, saving model to models/inception_rsna_checkpoint/rsna_1024_bias_weights_adam-lr-default-01-prc0.02-tp143.0-loss171825.52.hdf5
875/875 [==============================] - 31442s 36s/step - loss: 0.4467 - tp: 221.0000 - fp: 10115.0000 - tn: 17306.0000 - fn: 358.0000 - accuracy: 0.6260 - precision: 0.0214 - recall: 0.3817 - auc: 0.4996 - prc: 0.0209 - val_loss: 171825.5156 - val_tp: 143.0000 - val_fp: 6801.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.0206 - val_precision: 0.0206 - val_recall: 1.0000 - val_auc: 0.5000 - val_prc: 0.0206
Epoch 2/100
627/875 [====================>.........] - ETA: 2:22:53 - loss: 0.4414 - tp: 249.0000 - fp: 11719.0000 - tn: 7930.0000 - fn: 166.0000 - accuracy: 0.4076 - precision: 0.0208 - recall: 0.6000 - auc: 0.4891 - prc: 0.0195

In [ ]:
rm.plot_history(history)
ev.plot_history(history)

In [ ]:
plt.plot(history.history['loss'])
plt.title('Binary Cross Entropy loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

### Increase batch size from 1 x 32 to 2 x 32

In [15]:
BATCH_SIZE = 2*32
IMG_SIZE = (1024, 1024, 1)
RANDOM_STATE = 42 # the random state that used to create cv splits

BASEPATH = '/scratch/siads699w23_class_root/siads699w23_class/irinb/train'

train_gen, val_gen = rm.get_train_val_generators(
        train_img_ids,
        val_img_ids,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
    
        from_numpy=True,
        basepath=BASEPATH,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
    
        shuffle=True,
        normalize=(0, 1),
        feature_extractor='CNN',
        CNN_preprocess = keras.applications.inception_v3.preprocess_input
        )

In [19]:
model = rm.make_or_restore_model(
    model_layers,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1]),
    augment_layers=True,
    base_model=inception,
    trainable=True,
    loss_func = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics=METRICS,
    checkpoint_path='models/inception_rsna_checkpoint/adam'
)

model.summary()

Checkpoint path:  models/inception_rsna_checkpoint/adam
---Checkpoint folder already exists
Creating a new model...
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 1024, 1024, 3)     0         
                                                                 
 inception_v3 (Functional)   (None, 30, 30, 2048)      21802784  
                                                                 
 sequential (Sequential)     (None, 1)                 2049      
                                                                 
Total params: 21,804,833
Trainable params: 21,768,352
Non-trainable params: 36,481
_________________________________________________________________


In [ ]:
start = time.time()

EPOCH = 100

early_stopping = tf.keras.callbacks.EarlyStopping('val_loss', patience=3) # very slow training, never finishes
CHECKPOINT_PATH = 'models/inception_rsna_checkpoint/rsna_1024_bias_weights_batch64_adam-lr-default-{epoch:02d}-prc{val_prc:.2f}-tp{val_tp}-loss{val_loss:.2f}.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = CHECKPOINT_PATH,
    save_weights_only = True,
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    save_freq = 'epoch',
    verbose = 1
)

history = model.fit(
    train_gen,
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint],
    use_multiprocessing = True,
    workers = 32,
    validation_data=val_gen,
    class_weight=class_weight,
    verbose = 1
    )


stop = time.time()
print('>>> Training time: ', (stop - start))  

Epoch 1/100


2023-04-10 23:40:50.830362: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [2]
	 [[{{node Placeholder/_3}}]]


437/437 [==============================] - ETA: 0s - loss: 0.4420 - tp: 270.0000 - fp: 12722.0000 - tn: 14665.0000 - fn: 311.0000 - accuracy: 0.5340 - precision: 0.0208 - recall: 0.4647 - auc: 0.4983 - prc: 0.0207  

2023-04-11 08:35:52.122941: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 253.74802, saving model to models/inception_rsna_checkpoint/rsna_1024_bias_weights_batch64_adam-lr-default-01-prc0.02-tp0.0-loss253.75.hdf5
437/437 [==============================] - 33413s 76s/step - loss: 0.4420 - tp: 270.0000 - fp: 12722.0000 - tn: 14665.0000 - fn: 311.0000 - accuracy: 0.5340 - precision: 0.0208 - recall: 0.4647 - auc: 0.4983 - prc: 0.0207 - val_loss: 253.7480 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 6769.0000 - val_fn: 143.0000 - val_accuracy: 0.9793 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.0207
Epoch 2/100
263/437 [=================>............] - ETA: 3:40:04 - loss: 0.4319 - tp: 57.0000 - fp: 3463.0000 - tn: 13035.0000 - fn: 277.0000 - accuracy: 0.7778 - precision: 0.0162 - recall: 0.1707 - auc: 0.4816 - prc: 0.0182

### Freeze and save weights from epoch 1 which has the lowest validation loss for predictions

In [20]:
model.load_weights('models/inception_rsna_checkpoint/selected_models/rsna_1024_bias_weights_batch64_adam-lr-default-01-prc0.02-tp0.0-loss253.75.hdf5')

model.trainable = False
model.save('models/inception_rsna_checkpoint/selected_models/rsna_1024_batch64_adam_lr1e-3_epoch1_frozen.hdf5')

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 1024, 1024, 3)     0         
                                                                 
 inception_v3 (Functional)   (None, 30, 30, 2048)      21802784  
                                                                 
 sequential (Sequential)     (None, 1)                 2049      
                                                                 
Total params: 21,804,833
Trainable params: 0
Non-trainable params: 21,804,833
_________________________________________________________________
